In [1]:
from db_communication import db_communication
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import ipywidgets as widgets
from IPython.display import Javascript
def run_all_bellow(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

In [2]:
config = {
    'user': 'root',
    'password': 'FDS-apm1',
    'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
    'port': '3306',
    'database': 'Fahrradshop' 
}
my_db = db_communication(config)

Successfully connected.


In [3]:
customer_df = my_db.get_table('Kunde')
cluster_df = my_db.get_table('Merkmalcluster')
features_df = my_db.get_table('Merkmale')
order_df = my_db.get_table("Auftrag")
config_df = my_db.get_table("Konfiguration")

__Initialize widgets__

In [4]:
# customer login/create
rand_customer = customer_df.sample() #init
widget_customer_lst = [
    widgets.Text(
        value=rand_customer['Vorname'].values[0],
        placeholder='Type something',
        description='Vorname:',
        disabled=False
    ),
    widgets.Text(
        value=rand_customer['Name'].values[0],
        placeholder='Type something',
        description='Name:',
        disabled=False
    ),
    widgets.Text(
        value=rand_customer['Mail'].values[0],
        placeholder='Type something',
        description='E-Mail:',
        disabled=False
    )
]
options_radioButtons = widgets.RadioButtons(
    options=['Login', 'Create new account'],
    description='Options:',
    disabled=False
)
# configurator (choose features for each cluster)
widget_config_lst = []
for cluster_name,cluster in cluster_df.iterrows():
    features = features_df[features_df['Merkmalcluster']==int(cluster.values[0])]['Bezeichnung']
    widget = widgets.Dropdown(
        options=features,
        description=cluster.values[1],
        disabled=False
    )
    widget_config_lst.append(widget)

# Bike Configurator

In [5]:
# customer login/create
for widget in widget_customer_lst:
    display(widget)
display(options_radioButtons)
# configurator (choose features for each cluster)
print("Konfigurieren Sie Ihr Fahrrad:")
for widget in widget_config_lst:
    display(widget)
# run all below
button = widgets.Button(description="Auftrag abschließen")
button.on_click(run_all_bellow)
display(button)

Text(value='Clingerman', description='Vorname:', placeholder='Type something')

Text(value='Hilton', description='Name:', placeholder='Type something')

Text(value='hilton.clingerman@gmail.com', description='E-Mail:', placeholder='Type something')

RadioButtons(description='Options:', options=('Login', 'Create new account'), value='Login')

Konfigurieren Sie Ihr Fahrrad:


...

Dropdown(description='Sattel', options=('Standardsattel Herren', 'Standardsattel Damen', 'Trekkingsattel Herre…

Dropdown(description='Bremsen', options=('Felgenbremse Standard', 'Felgenbremse Trekking', 'Scheibenbremse Sta…

Dropdown(description='Beleuchtung', options=('Standard Batteriebeleuchtung', 'Komfort Batteriebeleuchtung', 'S…

Dropdown(description='Pedalen', options=('Standard Pedale', 'Trekking Pedale', 'Sport Pedale', 'Komfort Pedale…

Button(description='Auftrag abschließen', style=ButtonStyle())

## actions to fill the database

__Login / New Account__

In [ ]:
# for login -> check if customer exists and remind customer_id
if options_radioButtons.value == 'Login':
    customer_login = customer_df[(customer_df['Vorname']==widget_customer_lst[0].value) & 
                                (customer_df['Name']==widget_customer_lst[1].value) &
                                (customer_df['Mail']==widget_customer_lst[2].value)]
    if len(customer_login) != 0:
        print("Login erfolgreich!")
    else:
        raise ValueError(widget_customer_lst[0].value + " " + widget_customer_lst[1].value + " mit der E-Mail-Adresse "+
                         widget_customer_lst[2].value + " existiert nicht in der Kunden-Tabelle")
    customer_id = customer_login['KundenNr'].astype(int).values[0]

In [ ]:
# for create new account -> put text input into the database and remind customer_id
if options_radioButtons.value == 'Create new account':
    customer_id = customer_df['KundenNr'].astype(int).max() + 1
    customer_keys = customer_df.keys().values
    customer_values = customer_id, widget_customer_lst[1].value, widget_customer_lst[0].value, widget_customer_lst[2].value
    customer_dict = dict(zip(customer_keys,customer_values))
    my_db.insert_data('Kunde',customer_dict)
    # update local customer_df
    customer_df = my_db.get_table('Kunde')

__Add order__

In [ ]:
# after determining the customer_id, the order can be added
# values
order_id = order_df['AuftrNr'].max() + 1
today = datetime.today()
order_date = today.strftime("%Y-%m-%d")
delivery_date = (today + timedelta(days=14)).strftime("%Y-%m-%d")
#customer_id

order_keys = order_df.keys().values
order_values = order_id, order_date, delivery_date, customer_id
order_dict = dict(zip(order_keys,order_values))
my_db.insert_data('Auftrag',order_dict)
# update local customer_df
order_df = my_db.get_table('Auftrag')

__Add configuration__

In [ ]:
# after determining the order_id, the configuration can be added
for idx,widget in enumerate(widget_config_lst):
    config_id = order_id * 100 + idx
    #order_id
    feature_id = features_df[features_df['Bezeichnung']==widget.value]['MerkmalNr'].values[0]
    
    config_keys = config_df.keys().values
    config_values = config_id, order_id, feature_id
    config_dict = dict(zip(config_keys,config_values))
    my_db.insert_data('Konfiguration',config_dict)
    # update local customer_df
    config_df = my_db.get_table('Konfiguration')